In [87]:
import pandas as pd
import requests
from requests.exceptions import InvalidURL
import json
import ast
import math

In [88]:
df = pd.read_excel("controlled_substances.xlsx")

In [89]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES
0,"(1-(4-Fluorobenzyl)-1H-indol-3-yl)(2,2,3,3- te...",7014.0,I,N,FUB-144
1,"[3,2-c]-furazan-5α-androstan-17β-ol",4000.0,III,N,NaN
2,"[3,2-c]pyrazole-androst-4-en-17β-ol",4000.0,III,N,NaN
3,1-(1-Phenylcyclohexyl)pyrrolidine,7458.0,I,N,"PCPy, PHP, rolicyclidine"
4,1-(2-Phenylethyl)-4-phenyl-4-acetoxypiperidine,9663.0,I,Y,"PEPAP, synthetic heroin"
...,...,...,...,...,...
550,Zipeprol (1-methoxy-3[-4-(2-methoxy-2- phenyle...,9873.0,I,Y,NaN
551,Zolpidem,2783.0,IV,N,"Ambien, Ivadal, Stilnoct, Stilnox"
552,Zopiclone,2784.0,IV,N,Lunesta
553,Zuranolone,2420.0,IV,N,ZURZUVAE


In [90]:
df = df.drop(df[df["NARC"]=="N"].index).reset_index(drop=True)

In [91]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES
0,1-(2-Phenylethyl)-4-phenyl-4-acetoxypiperidine,9663.0,I,Y,"PEPAP, synthetic heroin"
1,1-Methyl-4-phenyl-4-propionoxypiperidine,9661.0,I,Y,"MPPP, synthetic heroin"
2,2'-Fluoro ortho-fluorofentanyl (N-(1-(2- fluor...,9855.0,I,Y,NaN
3,2-methoxy-N-(1-phenethylpiperidin-4-yl)-N- phe...,9825.0,I,Y,Methoxyacetyl fentanyl
4,3-Methylfentanyl,9813.0,I,Y,"China White, fentanyl"
...,...,...,...,...,...
192,Tramadol (2-[(dimethylamino)methyl]-1-(3- meth...,9752.0,IV,Y,Tramadol
193,Trimeperidine,9646.0,I,Y,Promedolum
194,"U-47700 (3,4-dichloro-N-[2-(dimethylamino)cycl...",9547.0,I,Y,U-47700
195,Valeryl fentanyl,9840.0,I,Y,N-(1-phenethylpiperidin-4-yl)-N-phenylpentanamide


In [92]:
import pandas as pd
import urllib.request
import json

# Function to get the rxcui for a single ingredient
def get_single_rxcui(item):
    item_encoded = urllib.parse.quote(item)
    url = f"https://rxnav.nlm.nih.gov/REST/rxcui.json?name={item_encoded}&search=2"

    try:
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read())
            rxcui = data.get('idGroup', {}).get('rxnormId')
            if rxcui:
                return ",".join(rxcui)
    except Exception as e:
        print(f"Error for ingredient {item}: {e}")

    return None

# Function to process an entry in the 'OTHER NAMES' column
def get_rxcui_for_other_names(entry):
    if pd.isna(entry):
        return None

    ingredients = entry.split(',')  # Split by comma for multiple ingredients
    rxcui_list = [get_single_rxcui(ingredient.strip()) for ingredient in ingredients]
    rxcui_list = [rxcui for rxcui in rxcui_list if rxcui is not None]  # Filter out None values

    return ','.join(rxcui_list) if rxcui_list else None

# Apply the function to each row in the DataFrame for the 'OTHER NAMES' column
df['rxcui_SUBSTANCE'] = df['SUBSTANCE'].apply(get_single_rxcui)
df['rxcui_OTHER_NAMES'] = df['OTHER NAMES'].apply(get_rxcui_for_other_names)


Error for ingredient : HTTP Error 400: Bad Request


In [93]:
df

,SUBSTANCE,CSCN,SCH,NARC,OTHER NAMES,rxcui_SUBSTANCE,rxcui_OTHER_NAMES
0,1-(2-Phenylethyl)-4-phenyl-4-acetoxypiperidine,9663.0,I,Y,"PEPAP, synthetic heroin",None,None
1,1-Methyl-4-phenyl-4-propionoxypiperidine,9661.0,I,Y,"MPPP, synthetic heroin",None,None
2,2'-Fluoro ortho-fluorofentanyl (N-(1-(2- fluor...,9855.0,I,Y,NaN,None,None
3,2-methoxy-N-(1-phenethylpiperidin-4-yl)-N- phe...,9825.0,I,Y,Methoxyacetyl fentanyl,None,None
4,3-Methylfentanyl,9813.0,I,Y,"China White, fentanyl",None,4337
...,...,...,...,...,...,...,...
192,Tramadol (2-[(dimethylamino)methyl]-1-(3- meth...,9752.0,IV,Y,Tramadol,None,10689
193,Trimeperidine,9646.0,I,Y,Promedolum,None,None
194,"U-47700 (3,4-dichloro-N-[2-(dimethylamino)cycl...",9547.0,I,Y,U-47700,None,None
195,Valeryl fentanyl,9840.0,I,Y,N-(1-phenethylpiperidin-4-yl)-N-phenylpentanamide,None,None


In [95]:
# Remove rows where both 'rxcui_SUBSTANCE' and 'rxcui_OTHER_NAMES' are None or empty
df = df.dropna(subset=['rxcui_SUBSTANCE', 'rxcui_OTHER_NAMES'], how='all')


In [97]:
df.reset_index(drop=True, inplace=True)

In [99]:
df.to_excel("controlled_substances_opioids.xlsx", index=False)